## Import packages

In [77]:
# preprocessing
import pandas as pd
import numpy as np
import datetime as dt

# viz
import altair as alt
alt.data_transformers.enable("vegafusion")
import matplotlib.pyplot as plt


## Loading dataset

In [57]:
# csv_url = 'https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/business-licences/exports/csv?lang=en&timezone=America%2FLos_Angeles&use_labels=true&delimiter=%3B'
# business = pd.read_csv(csv_url, delimiter = ';')

# It takes a while to load data from the url, so... here's the shortcut!
# Just download the file above to your local machine, and put the file in the data folder
business = pd.read_csv('data/business-licences.csv', delimiter = ';')

/var/folders/x5/4d64q7td2l59kf73lvykgkq40000gn/T/ipykernel_56756/3272950823.py:6: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  business = pd.read_csv('data/business-licences.csv', delimiter = ';')


## Preprocessing

### Cleaning data
- Drop rows where `ExpiredDate` and `IssuedDate` are NA.
- Transform `ExpiredDate` and `IssuedDate` to date.
- Calculate the survival interval of each company, which is the difference between the maximum of ExpiredDate and the minimum of IssuedDate.
- Keep only the newest issued record of each company.
- Filter to keep those records where the latest `ExpiredDate` is before or equal to year 2022 because for those licenses issued in year 2023, the dafault `ExpiredDate` are `2023-12-31` and we cannot know whether it would survive until then.

In [59]:
# Drop rows where ExpiredDate and IssuedDate are NA
business = business.dropna(subset = ["ExpiredDate", "IssuedDate"])

# Transform ExpiredDate and IssuedDate to date
business[["ExpiredDate", "IssuedDate"]] = business[["ExpiredDate", "IssuedDate"]].apply(pd.to_datetime, utc=True)
business['ExpiredDate'] = business['ExpiredDate'].dt.date
business['IssuedDate'] = business['IssuedDate'].dt.date

# Calculate the survival interval of each company
business['survival_days'] = (business.groupby('BusinessName')['ExpiredDate'].transform('max')-
                            business.groupby('BusinessName')['IssuedDate'].transform('min'))
business['survival_days'] = pd.to_timedelta(business['survival_days']).dt.days

# Keep only the newest issued record of each company
business.sort_values(by='ExpiredDate', ascending=True)
business = business.drop_duplicates(subset='BusinessName', keep='last')

# Filter to keep those records where the latest `ExpiredDate` is before or equal to year 2022.
business = business[business['ExpiredDate'] <= dt.date(2022, 12, 31)]

### Response Variable for Classification: survival_status

In [66]:
survival_threshold = 730
business['survival_status'] = business['survival_days'] >= survival_threshold
business["survival_status"] = business["survival_status"].astype(int)
business

,FOLDERYEAR,LicenceRSN,LicenceNumber,LicenceRevisionNumber,BusinessName,BusinessTradeName,Status,IssuedDate,ExpiredDate,BusinessType,...,Country,PostalCode,LocalArea,NumberofEmployees,FeePaid,ExtractDate,Geom,geo_point_2d,survival_days,survival_status
21,13,1786109,13-166693,0,David Andrew Goodfellow (David Goodfellow),Bramblebutt Productions,Issued,2012-12-22,2013-12-31,Entertainment Services,...,CA,NaN,Mount Pleasant,1.0,129.0,2019-07-21T13:49:06-07:00,NaN,NaN,374,0
26,13,1786136,13-166720,0,William David McKnight (William McKnight),NaN,Issued,2012-12-28,2013-12-31,Entertainment Services,...,CA,NaN,Grandview-Woodland,0.0,129.0,2019-07-21T13:49:06-07:00,NaN,NaN,368,0
27,13,1786138,13-166722,0,(Thomas MacDonald),Mr. MacDonald Music,Issued,2013-01-30,2013-12-31,Entertainment Services,...,CA,NaN,Grandview-Woodland,0.0,169.0,2019-07-21T13:49:06-07:00,NaN,NaN,335,0
29,13,1786144,13-166728,0,(Janet Morrison),Janet Love Morrison Editing,Issued,2012-12-22,2013-12-31,Entertainment Services,...,CA,NaN,West End,1.0,129.0,2019-07-21T13:49:06-07:00,NaN,NaN,374,0
36,13,1786165,13-166749,0,Laura Rose Martin Barreca (Laura Barreca),NaN,Issued,2013-06-14,2013-12-31,Entertainment Services,...,CA,NaN,Mount Pleasant,0.0,151.0,2019-07-21T13:49:06-07:00,NaN,NaN,200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646924,21,3724155,21-141639,0,Six Oh Four Printing Ltd,Made in Print,Issued,2020-12-01,2021-12-31,Printing Services,...,CA,V6B 1C2,Downtown,5.0,155.0,2023-11-01T02:38:57-07:00,"{""coordinates"": [-123.114538756358, 49.2766297...","49.2766297368584, -123.114538756358",2348,1
646928,21,3724163,21-141647,0,Saing Chun Derek Chan (Saing Chan),RSVP Wedding Invitations & Printing,Issued,2021-02-10,2021-12-31,Printing Services,...,CA,V6A 1H9,Strathcona,1.0,195.0,2023-11-01T02:38:57-07:00,"{""coordinates"": [-123.08364749238, 49.28233634...","49.2823363433113, -123.08364749238",3672,1
646935,21,3724233,21-141717,0,Ricardo W Thaller (Ricardo Thaller),NaN,Issued,2021-04-07,2021-12-31,Plumber & Gas Contractor,...,CA,NaN,South Cambie,1.0,390.0,2023-11-01T02:38:57-07:00,NaN,NaN,3220,1
646950,21,3724334,21-141818,0,Vital Plumbing And Heating Inc,NaN,Inactive,2021-02-08,2021-12-31,Plumber & Gas Contractor,...,CA,NaN,West End,0.0,195.0,2023-11-01T02:38:57-07:00,NaN,NaN,1255,1


## EDA & Visualization

In [68]:
business.describe()

,FOLDERYEAR,LicenceRSN,LicenceRevisionNumber,NumberofEmployees,FeePaid,survival_days,survival_status
count,50891.000000,5.089100e+04,50891.000000,50891.000000,50547.000000,50891.000000,50891.000000
mean,16.341730,2.677001e+06,0.022322,4.377925,234.435544,1179.374015,0.568077
std,2.948827,6.952117e+05,0.153215,28.902300,670.168871,959.342841,0.495349
min,13.000000,1.771189e+06,0.000000,0.000000,1.000000,-147.000000,0.000000
25%,14.000000,2.132468e+06,0.000000,0.000000,129.000000,379.000000,0.000000
50%,16.000000,2.570413e+06,0.000000,1.000000,151.000000,859.000000,1.000000
75%,19.000000,3.254372e+06,0.000000,2.000000,189.000000,1805.000000,1.000000
max,22.000000,4.201432e+06,3.000000,2016.000000,33958.000000,3697.000000,1.000000


In [132]:
business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50891 entries, 21 to 646957
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   FOLDERYEAR             50891 non-null  int64  
 1   LicenceRSN             50891 non-null  int64  
 2   LicenceNumber          50891 non-null  object 
 3   LicenceRevisionNumber  50891 non-null  int64  
 4   BusinessName           50891 non-null  object 
 5   BusinessTradeName      24153 non-null  object 
 6   Status                 50891 non-null  object 
 7   IssuedDate             50891 non-null  object 
 8   ExpiredDate            50891 non-null  object 
 9   BusinessType           50891 non-null  object 
 10  BusinessSubType        31715 non-null  object 
 11  Unit                   12816 non-null  object 
 12  UnitType               12805 non-null  object 
 13  House                  24535 non-null  object 
 14  Street                 24536 non-null  object 
 15  City 

### survival_status value_counts

In [95]:
business['survival_status'].value_counts()

survival_status
1    28910
0    21981
Name: count, dtype: int64

### survival_status rate v.s. BusinessType

In [120]:
busi_type_rate = business[['BusinessType', 'survival_status']]
busi_type_rate = busi_type_rate.groupby('BusinessType').agg(count=('survival_status', 'size'), survival_rate=('survival_status', 'mean')).reset_index()

alt.Chart(busi_type_rate).mark_point().encode(
    x=alt.X('survival_rate'),
    y=alt.Y('count'),
    tooltip='BusinessType'
)

alt.Chart(...)

### survival_status rate v.s. City
- Limited in Vancouver, for most of the businesses in this dataset are in Vancouver, BC
- Most of the businesses are located in Downtown 
- There are no significant differences among the survival rates across areas.

In [138]:
business.groupby('Province').size().reset_index()

,Province,0
0,78,1
1,AB,101
2,AL,1
3,AZ,1
4,Ab,1
5,BC,50308
6,British Columbia,1
7,CA,73
8,CO,3
9,CT,2


In [144]:
business[business['Province'] == 'BC'].groupby('City').size().reset_index().sort_values(by=0, ascending = False)

,City,0
166,Vancouver,41343
18,Burnaby,1652
155,Surrey,1555
131,Richmond,931
107,North Vancouver,881
...,...,...
75,Lantzville,1
74,Langley Township,1
72,Langey,1
70,Ladysmith,1


In [129]:
city_rate = business[business['Province'] == 'BC'][['City', 'survival_status']]
city_rate = city_rate.groupby('City').agg(count=('survival_status', 'size'), survival_rate=('survival_status', 'mean')).reset_index()

alt.Chart(city_rate).mark_point().encode(
    x=alt.X('survival_rate'),
    y=alt.Y('count'),
    tooltip='City'
)

alt.Chart(...)

In [145]:
local_area_rate = business[business['City'] == 'Vancouver'][['LocalArea', 'survival_status']]
local_area_rate = local_area_rate.groupby('LocalArea').agg(count=('survival_status', 'size'), survival_rate=('survival_status', 'mean')).reset_index()

alt.Chart(local_area_rate).mark_point().encode(
    x=alt.X('survival_rate'),
    y=alt.Y('count'),
    tooltip='LocalArea'
)

alt.Chart(...)

### survival_status v.s. NumberofEmployees
As last part, we focus in the Vancouver city only.
- There seems no specific threshold in regards of NumberofEmployees. But we can still use LogisticRegression to see whether it does.

In [148]:
alt.Chart(business[business['City'] == 'Vancouver'][['NumberofEmployees', 'survival_status']]).mark_point().encode(
    x=alt.X('NumberofEmployees'),
    y=alt.Y('survival_status'),
)

alt.Chart(...)